In [ ]:
import asyncio
import nest_asyncio
import json
import unittest
from os import path, getcwd

import funman.api.client as client
from funman.api.api import app
from funman.api.server import Server, ServerConfig

API_BASE_PATH = path.join(os.getcwd(), "..")
API_SERVER_HOST = "0.0.0.0"
API_SERVER_PORT = 8190
SERVER_URL = f"http://{API_SERVER_HOST}:{API_SERVER_PORT}"
OPENAPI_URL = f"{SERVER_URL}/openapi.json"
CLIENT_NAME = "funman-api-client"

In [ ]:
# Start API Server

server = Server(
            config=ServerConfig(
                app,
                host=API_SERVER_HOST,
                port=API_SERVER_PORT,
                log_level="info",
            )
        )
# loop = asyncio.get_event_loop()
# loop.create_task(server.serve())
nest_asyncio.apply()
server.run_in_thread()

In [ ]:
# Make API client

client.make_client(
    API_BASE_PATH, openapi_url=OPENAPI_URL, client_name=CLIENT_NAME
)
from funman_api_client import Client


funman_client = Client(SERVER_URL, timeout=None)

In [12]:
# Setup API client request

from funman_api_client.api.default import (
    solve_consistency_solve_consistency_put,
)
from funman_api_client.models import (
    BodySolveConsistencySolveConsistencyPut,
    ConsistencyScenario,
    ConsistencyScenarioResult,
    FUNMANConfig
)

In [13]:
# Construct request


bilayer_json =     {"Wa":[{"influx":1,"infusion":2},
			 {"influx":2,"infusion":3}],
 "Win":[{"arg":1,"call":1},
 				{"arg":2,"call":1},
 				{"arg":2,"call":2}],
 "Box":[{"parameter":"beta"},
 				{"parameter":"gamma"}],
 "Qin":[{"variable":"S"},
 				{"variable":"I"},
 				{"variable":"R"}],
 "Qout":[{"tanvar":"S'"},
 				 {"tanvar":"I'"},
 				 {"tanvar":"R'"}],
 "Wn":[{"efflux":1,"effusion":1},
 			 {"efflux":2,"effusion":2}]}


infected_threshold = 130
init_values = {"S": 9998, "I": 1, "R": 1}


response = asyncio.run(
    solve_consistency_solve_consistency_put.asyncio_detailed(
        client=funman_client,
        json_body=BodySolveConsistencySolveConsistencyPut(
            ConsistencyScenario.from_dict(
                {
                    "model": {
                        "init_values": init_values,
                        "bilayer": {"json_graph": bilayer_json},
                    },
                    "query": {
                        "variable": "I",
                        "ub": infected_threshold,
                        "at_end": False,
                    },
                }
            ),
            FUNMANConfig.from_dict(
                            {"tolerance": 1.0e-8, "number_of_processes": 1}
                        ),
        ),
    )
)

result = ConsistencyScenarioResult.from_dict(
    src_dict=json.loads(response.content.decode())
)
result.to_dict()

INFO:     127.0.0.1:33414 - "PUT /solve/consistency HTTP/1.1" 200 OK


{'scenario': {'model': {'bilayer': {'json_graph': {'Wa': [{'influx': 1,
       'infusion': 2},
      {'influx': 2, 'infusion': 3}],
     'Win': [{'arg': 1, 'call': 1},
      {'arg': 2, 'call': 1},
      {'arg': 2, 'call': 2}],
     'Box': [{'parameter': 'beta'}, {'parameter': 'gamma'}],
     'Qin': [{'variable': 'S'}, {'variable': 'I'}, {'variable': 'R'}],
     'Qout': [{'tanvar': "S'"}, {'tanvar': "I'"}, {'tanvar': "R'"}],
     'Wn': [{'efflux': 1, 'effusion': 1}, {'efflux': 2, 'effusion': 2}]}},
   'measurements': None,
   'identical_parameters': [],
   'init_values': {'S': 9998.0, 'I': 1.0, 'R': 1.0},
   'parameter_bounds': {}},
  'query': {'variable': 'I', 'ub': 130.0, 'at_end': False}},
 'consistent': {'S_1': 0.125,
  'R_1': 9870.0,
  'gamma': 9869.0,
  'I_1': 129.875,
  'I_0': 1.0,
  'R_2': 1291606.375,
  'S_2': -16.109172029718444,
  'I_2': -1281590.2658279703,
  'gamma_1': 9869.0,
  'S_0': 9998.0,
  'gamma_0': 9869.0,
  'R_0': 1.0,
  'beta_0': 0.9999874974994999,
  'beta': 0.99